In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import time
                                         
# Load data
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))
y = np.array(y)
X = X / 255.0  # Normalize

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = f"{conv_layer}-conv-nodes-{layer_size}-dense-{dense_layer}-{int(time.time())}"
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
            # Define model properly
            model = Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten()) # 3D feature maps converted to 1D feature vector
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
                
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
            
            # Train model with Validation Set
            model.fit(X, y, batch_size=32, epochs=5, validation_split=0.3, callbacks=[tensorboard])

model.save('64x3-CNN.keras')


Epoch 1/5
546/546 ━━━━━━━━━━━━━━━━━━━━ 37s 59ms/step - accuracy: 0.5733 - loss: 0.6746 - val_accuracy: 0.6907 - val_loss: 0.5926
Epoch 2/5
546/546 ━━━━━━━━━━━━━━━━━━━━ 32s 58ms/step - accuracy: 0.7194 - loss: 0.5565 - val_accuracy: 0.7401 - val_loss: 0.5174
Epoch 3/5
546/546 ━━━━━━━━━━━━━━━━━━━━ 35s 64ms/step - accuracy: 0.7693 - loss: 0.4784 - val_accuracy: 0.7746 - val_loss: 0.4749
Epoch 4/5
546/546 ━━━━━━━━━━━━━━━━━━━━ 31s 57ms/step - accuracy: 0.8030 - loss: 0.4334 - val_accuracy: 0.7886 - val_loss: 0.4522
Epoch 5/5
546/546 ━━━━━━━━━━━━━━━━━━━━ 32s 58ms/step - accuracy: 0.8174 - loss: 0.3954 - val_accuracy: 0.7786 - val_loss: 0.4750


In [4]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (24946, 50, 50, 1)
y shape: (24946,)


In [11]:
import cv2
import tensorflow as tf
CATEGORIES = ["Cat", "Dog"]

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE, 1)

model = tf.keras.models.load_model("64x3-CNN.keras")

In [17]:
prediction = model.predict([prepare('dog.png')])

print(CATEGORIES[int(prediction[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Dog


In [22]:
prediction = model.predict([prepare('cat.jpg')])

print(CATEGORIES[int(prediction[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Cat
